<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/Kaggle_Contradictory%2C_My_Dear_Watson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contradictory, My Dear Watson

Detecting contradiction and entailment in multilingual text **using TPUs**

- TPU : 구글에서 2016년 5월에 발표한 데이터 분석 및 딥러닝용 하드웨어이다. 벡터/행렬연산의 병렬처리에 특화되어 있으며 넘사벽급의 전성비를 자랑한다.

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!pip install transformers

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  37343      0 --:--:-- --:--:-- --:--:-- 37343
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.9.0+cu102
     |████████████████████████████████| 57 kB 2.7 MB/s 
Uninstalling torch-1.9.0+cu102:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.272 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatibl

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import torch_xla
import torch_xla.core.xla_model as xm

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold

import transformers
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW

import warnings
warnings.filterwarnings('ignore')

print(torch.__version__)

1.6.0a0+bf2bbd9


# Bert model 

---



==> Bidirectional(양방향 모델) : 왼쪽에서 오른쪽으로, 오른쪽에서 왼쪽으로 모두 텍스트를 읽을 수 있음을 의미합니다.

In [3]:
# Bert Vocabulary

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
len(tokenizer.vocab)

30522

In [10]:
# token의 구성
bert_vocab = tokenizer.vocab

print(bert_vocab['[CLS]'])
print(bert_vocab['[SEP]'])
print(bert_vocab['[PAD]']) # padding=0

# 문자열 토큰화
print(bert_vocab['hello'])
print(bert_vocab['world'])

bert_keys = []

for token in tokenizer.vocab.keys():
  bert_keys.append(token)

print(bert_keys[101])
print(bert_keys[102])
print(bert_keys[0])

print(bert_keys[7592])
print(bert_keys[2088])

101
102
0
7592
2088
[CLS]
[SEP]
[PAD]
hello
world


In [11]:
# Bert model 구성요소

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased',
                                                      num_labels=3,  # 결과 라벨 수
                                                      output_attentions=False,
                                                      output_hidden_states=False)

outputs = model(input_ids=b_input_ids,
                token_type_ids=b_token_type_ids,
                attention_mask=b_attention_mask,
                labels=b_labels)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

NameError: ignored

1. input_ids
 - 토큰으로 표시되는 문장(토큰)
 
 [CLS] : classifier token, vlaue:101

 [SEP] : separator token, value:102

 [PAD] : padding token, value:0

  For one sentence as input:
    - [CLS] ...word tokens... [SEP]

  For two sentences as input:
    - [CLS] ...sentence1 tokens... [SEP]..sentence2 tokens... [SEP]

ex) [101[CLS], 7592, 2045, 1012,  102[SEP],    0,    0,    0,    0,    0]

2. token_type_ids
 - 입력의 일부가 되어야 하는 두 문장이 있을 때 사용
 - [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
 - 0은 첫번째 sentence 식별
 - 1은 두번째 sentence 식별
 - 오른쪽 0은 padding

3. attantion_mask
 - attention_mask의 길이는 input_ids의 길이와 같아야 한다.
 - 모델에 input_ids의 어떤 토큰이 워드이고 어떤 토큰이 패딩인지 알려준다.

ex)
 - Tokens: [101, 7592, 2045, 1012,  102,    0,    0,    0,    0,    0]
 - Attention mask: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

4. labels
 - 각 행의 label(target)
 - ex) [0, 2, 0, 1, 2, 0, 3, 1]

---

# how to use tokenizer to auto format input data

In [13]:
# encode
max_length = 10
sentence1 = 'hello there.'

encoded_dict = tokenizer.encode_plus(
    sentence1,  # text to tokens
    add_special_tokens=True,  # [CLS], [SEP]를 추가
    max_length=max_length,
    truncation=True,  # # Pad or truncate.
    return_attention_mask=True,
    return_tensors='pt'
)

encoded_dict

{'input_ids': tensor([[ 101, 7592, 2045, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [15]:
input_ids = encoded_dict['input_ids'][0]
token_type_ids = encoded_dict['token_type_ids'][0]
attention_mask = encoded_dict['attention_mask'][0]

print(input_ids)
print(token_type_ids)
print(attention_mask)

tensor([ 101, 7592, 2045, 1012,  102])
tensor([0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1])


In [18]:
# decode
a = tokenizer.decode(input_ids,
                     skip_special_tokens=False)

b = tokenizer.decode(input_ids,
                     skip_special_tokens=True)

print(a)
print(b)

[CLS] hello there. [SEP]
hello there.


# XLM-RoBERTa Vocabulary

- XLM은 교차 언어 모델을 의미. 

- XLM-Roberta(XLM-R)는 다국어 BERT를 능가하는 사전 훈련된 **다국어 모델**이다.
 - XLM-R이 훨씬 더 많은 데이터를 사용하여 교육을 받았기 때문입니다. (XLM-R은 또한 100개의 언어로 train 되었다.)

1. input_ids
 - BERT의 토큰화와 비슷
    - bos_token_id (s) : 0
    - eos_token_id (/s) : 2
    - sep_token_id (/s) : 2
    - pad_token_id (pad) : 1

 - For one sentence as input:
    - (s) ...word tokens... (/s)

 - For two sentences as input:<br>
    - (s) ...sentence1 tokens... (/s)(/s)..sentence2 tokens... (/s)

 - padding은 1
    - ex)[[0, 35378, 2685, 5, 2, 1, 1, 1, 1, 1]]


2. token_type_ids
 - XLM-RoBERTa는 token_type_ids를 사용하지 않는다.

3. attention_mask
 - BERT와 같다.

4. labels
 - target 레이블(BERT와 같다.)


In [3]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [2]:
model_type='xlm-roberta-base'
tokenizer = XLMRobertaTokenizer.from_pretrained(model_type)

In [3]:
print(tokenizer.vocab_size)  # check vocab size
tokenizer.special_tokens_map # what are the special tokens

250002


{'bos_token': '<s>',
 'cls_token': '<s>',
 'eos_token': '</s>',
 'mask_token': '<mask>',
 'pad_token': '<pad>',
 'sep_token': '</s>',
 'unk_token': '<unk>'}

In [ ]:
# xlm-roberta-base model

model_type = 'xlm-roberta-base'

model = XLMRobertaForSequenceClassification.from_pretrained(model_type,
                                                            num_labels=3)

In [6]:
# encode
max_len = 10

sentence1 = 'hello there.'

encoded_dict = tokenizer.encode_plus(sentence1,
                                     add_special_tokens=True,
                                     max_length=max_len,
                                     truncation=True,
                                     return_attention_mask=True,
                                     return_tensors='pt')

encoded_dict
# input_ids, attention_mask로 구성 -- Bert는 input_ids, attention_mask, token_type_ids 구성

{'input_ids': tensor([[    0, 33600,    31,  2685,     5,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [7]:
input_ids = encoded_dict['input_ids'][0]
att_mask = encoded_dict['attention_mask'][0]

print(input_ids)
print(att_mask)

tensor([    0, 33600,    31,  2685,     5,     2])
tensor([1, 1, 1, 1, 1, 1])


In [9]:
# decode
a = tokenizer.decode(input_ids,
                     skip_special_tokens=False)  # 'bos_token': '<s>', 'cls_token': '<s>', 'eos_token': '</s>', 'mask_token': '<mask>', 'pad_token': '<pad>', 'sep_token': '</s>', 'unk_token': '<unk>' 등

b = tokenizer.decode(input_ids,
                     skip_special_tokens=True)

print(a)
print(b)

<s> hello there.</s>
hello there.


# Contradictory, My Dear Watson 실습 Start

In [2]:
train = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/contradictory-my-dear-watson/contradictory-my-dear-watson-train.csv')
test = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/contradictory-my-dear-watson/contradictory-my-dear-watson-test.csv')
submit = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/contradictory-my-dear-watson/contradictory-my-dear-watson-sample_submission.csv')
train.shape, test.shape, submit.shape

((12120, 6), (5195, 5), (5195, 2))

In [15]:
train.head()
# XLM-RoBERTa 쓰기 딱 좋구만...

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [3]:
df = shuffle(train)

sk = StratifiedKFold(n_splits=5, shuffle=True, random_state=1024)

y_train = df['label']

fold_list = list(sk.split(df, y_train))

train_list = []
val_list = []

for i, fold in enumerate(fold_list):
  df_train = df[df.index.isin(fold[0])]
  df_val = df[df.index.isin(fold[1])]

  train_list.append(df_train)
  val_list.append(df_val)

print(f'train length : {len(train_list)}')
print(f'val length : {len(val_list)}')

train length : 5
val length : 5


In [27]:
# 위에 하고 같음
list_train = []
list_val = []
for i, (train_index, val_index) in enumerate(sk.split(df, y_train)):
  train_df, val_df = df.iloc[train_index], df.iloc[val_index]

  list_train.append(train_df)
  list_val.append(val_df)

In [22]:
train_list[0]

,id,premise,hypothesis,lang_abv,language,label
10505,fbfaa58bc0,Pick up a map from the tourist office here and...,The people at the tourist office are friendly ...,en,English,1
3144,fb8cad0a7a,"और उह्। उसने खींचा, उसने इसे बाहर निकाल दिया, ...",वो खुद सा महसूस करने लगा है जबसे उस सोच ने बाह...,hi,Hindi,0
8421,c6e980fab1,"Това означава, че всички молекулярни елементи ...",Не можете да добавяте неща с постоянно темпо.,bg,Bulgarian,2
9757,46ef676d38,"But to you, who know the truth, I propose to r...",Two of the thin pages stuck together as he lea...,en,English,1
7676,b9b7aa3dd5,The event is the definition of a crowd pleaser...,Both Santa Claus and the Rockettes with appear...,en,English,0
...,...,...,...,...,...,...
4674,6a3c753fc0,Albino Alligator (Miramax).,Albino alligators are very rare in wild.,en,English,1
851,8f99bd3225,"Then, all the time, it was in the spill vase i...",You mean to say it was stupid for us to look s...,en,English,1
10420,c4206db6f0,यह निजी सहायता और हमारे कानून विद्यालय के लिए ...,हमारा विद्यालय (लॉ स्कूल) केवल निजी तौर पर लगा...,hi,Hindi,2
10083,cb81a587e0,They encourage the view that there's nothing--...,There is nothing a few completely visible plan...,en,English,2


In [23]:
val_list[0]

,id,premise,hypothesis,lang_abv,language,label
7317,9831ebb202,"Around the corner is the huge, domed, Neo-Clas...",The Pantheon is from the Classical era.,en,English,2
8827,f15a6b4d68,But there's plenty more.,There is not a sufficient amount.,en,English,2
9241,d204eb65a9,"उदाहरण के लिए, एक संस्था जिसको हमने अध्ययन किय...","हालांकि दोनों के विलय हुए थे, फिर भी दोनों कंप...",hi,Hindi,2
7511,f769a29351,Napoleon attackierte und zerstörte Kataloniens...,Napoleon war verantwortlich für die Zerstörung...,de,German,0
6870,db58070c65,What and who will they tax?,We know all about their tax plan.,en,English,2
...,...,...,...,...,...,...
1542,d817ff59b2,既然我们独立了，请帮助我们加大和你的水平差距 。,我们希望你能在我们独立时帮助我们。,zh,Chinese,0
8779,abe1ad9610,yeah well i'm a hot weather person i'm i can t...,"The cold weather makes me sick, thus I am not ...",en,English,1
10019,7ad438401a,"Tôi sẽ không giam bà nữa, thưa bà.","Thưa bà, thật không công bằng khi giam giữ bà.",vi,Vietnamese,1
8994,515a9afb8f,The strychnine had been found in a drawer in t...,There is a drawer in the prisoner's quarters.,en,English,0


# 토큰화, Dataset, Dataloader

In [4]:
MODEL_TYPE = 'bert-base-multilingual-uncased'

NUM_FOLDS = 5

# Saving 5 TPU models will exceed the 4.9GB disk space.
# Therefore, will will only train on 3 folds.
NUM_FOLDS_TO_TRAIN = 3 

L_RATE = 1e-5
MAX_LEN = 256
NUM_EPOCHS = 3
BATCH_SIZE = 32

In [5]:
device = xm.xla_device()
print(device)

xla:1


In [6]:
# bert-base-multilingual-uncased 토큰
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE,
                                          do_lower_case=True)

In [7]:
# Dataset으로 tokenizer까지
class CompDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    target = torch.tensor(self.df_data.loc[index, 'label'])

    sample = (input_ids, attention_mask, token_type_ids, target)
    return sample

  def __len__(self):
    return len(self.df_data)

class TestDataset(Dataset):
  def __init__(self, df):
    self.df_data = df



  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    sample = (input_ids, attention_mask, tokens_type_ids)

    return sample

  def __len__(self):
    return len(self.df_data)

In [34]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           # Sentences to encode.
                    add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                    max_length = MAX_LEN,           # Pad or truncate all sentences.
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )  
        
        # These are torch tensors already.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        token_type_ids = encoded_dict['token_type_ids'][0]
        
        # Convert the target to a torch tensor
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, token_type_ids, target)


        return sample


    def __len__(self):
        return len(self.df_data)
    
    
    
    
    

class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           # Sentence to encode.
                    add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                    max_length = MAX_LEN,           # Pad or truncate all sentences.
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )
        
        # These are torch tensors already.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        token_type_ids = encoded_dict['token_type_ids'][0]
               

        sample = (padded_token_list, att_mask, token_type_ids)


        return sample


    def __len__(self):
        return len(self.df_data)

In [29]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [14]:
# num_workers 쓰기 위한 cpu_count()
NUM_CORES = os.cpu_count()

NUM_CORES

40

In [49]:
train = CompDataset(df_train)
valli = CompDataset(df_val)
test = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=4)

val_dataloader = torch.utils.data.DataLoader(valli,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True,
                                             num_workers=4)

test_dataloader = torch.utils.data.DataLoader(test,
                                              batch_size=BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=4)

len(train_dataloader), len(val_dataloader), len(test_dataloader)

(303, 76, 163)

# Define the Model(BertForSequenceClassification)

In [16]:
# model 불러오기(BertForSequnceClassification)
model = BertForSequenceClassification.from_pretrained(MODEL_TYPE,
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

# Test the Model

In [50]:
train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=8,
                                               shuffle=True,
                                               num_workers=1)

batch = next(iter(train_dataloader))

b_input_ids = batch[0].to(device)
b_input_mask = batch[1].to(device)
b_token_type_ids = batch[2].to(device)
b_labels = batch[3].to(device)

b_input_ids.size(), b_input_mask.size(), b_token_type_ids.size(), b_labels.size()

(torch.Size([8, 256]),
 torch.Size([8, 256]),
 torch.Size([8, 256]),
 torch.Size([8]))

# 모델의 output 탐구



In [51]:
# output 생성
outputs = model(b_input_ids, 
                token_type_ids=b_token_type_ids, 
                attention_mask=b_input_mask,
                labels=b_labels)

In [52]:
 # 하나의 튜플 안에 (loss, predicts)
 len(outputs)

2

In [53]:
# loss
outputs[0]

tensor(1.2436, device='xla:1', grad_fn=<NllLossBackward>)

In [54]:
# predicts
outputs[1]

tensor([[-0.3263,  1.0046, -0.0553],
        [-0.1865,  0.6285, -0.0638],
        [-0.1634,  0.5392, -0.0733],
        [-0.1848,  0.6236, -0.0805],
        [ 0.0098,  0.1407, -0.0731],
        [-0.2627,  0.9345, -0.0445],
        [-0.1448,  0.5292, -0.0558],
        [-0.0328,  0.2503, -0.0908]], device='xla:1', grad_fn=<AddmmBackward>)

In [55]:
preds = outputs[1].detach().cpu().numpy()

y_true = b_labels.detach().cpu().numpy()
y_pred = np.argmax(preds, axis=1)

y_pred

array([1, 1, 1, 1, 1, 1, 1, 1])

In [56]:
acc = accuracy_score(y_true=y_true, y_pred=y_pred)

acc

0.25

# Train the Model

In [37]:
seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

fold_val_acc_list = []
for i in range(0, NUM_FOLDS):
  fold_val_acc_list.append([])

# for each fold
for epoch in range(0, NUM_EPOCHS):
  print("\nNum folds used for training:", NUM_FOLDS_TO_TRAIN)
  print(f'=========== Epoch {epoch+1} / {NUM_EPOCHS} ===========')

  num_folds = lend(train_list)

  epoch_acc_scores_list = []

  for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
    print('\n== Fold Model', fold_index)

    if epoch == 0:
      model = BertForSequenceClassification.from_pretrained(MODEL_TYPE,
                                                            num_labels=3,
                                                            output_attention=False,
                                                            output_hidden_states=False)
      
      model.to(device)
      optimizer = AdamW(model.parameters(),
                        lr=L_RATE,
                        eps=1e-8)
    
    else:
      path_model = 'model_' + str(fold_index) + '.bin'
      model.load_state_dict(torch.load(path_model))

      model.to(device) 
    
    # Data 불러오기
    df_train = train_list[fold_index]
    df_val = val_list[fold_index]

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)

    train_data = CompDataset(df_train)
    val_data = CompDataset(df_val)

    train_dataloader = torch.utils.data.DataLoader(train_data,
                                                   batch_size=BATCH_SIZE,
                                                   shuffle=True,
                                                   num_workers=4)
    val_dataloader = torch.utils.data.DataLoader(val_data,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=True,
                                                 num_workers=4)
    # Training
    stacked_val_labels=[]
    targets_list=[]

    print('Training...')
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss=0

    for i, batch in enumerate(train_dataloader):
      train_status = 'Batch' + str(i+1) + 'of' + str(len(train_dataloader))

      print(train_status, end='\r')

      input_ids = batch[0].to(device)
      attention_mask = batch[1].to(device)
      token_type_ids = batch[2].to(device)
      labels = batch[3].to(device)

      model.zero_grad()

      outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                      token_type_ids=token_type_ids, labels=labels)
      loss=outputs[0]
      total_train_loss += loss.item()
      
      optimizer.zero_grad()
      loss.backward()
      
      # Clip the norm of the gradients to 1.0.
      # This is to help prevent the "exploding gradients" problem.
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Optimizer for GPU
      # opimizer.step()

      # Optimizer for TPU
      xm.optimizer_step(optimizer=optimizer, barrier=True)

      print('Train loss:', total_train_loss)

      
      # Validation
      print('\nValidation...')
      model.eval()
      
      torch.set_grad_enabled(False)
      total_val_loss=0

      for j, val_batch in enumerate(val_dataloader):
        val_status = 'Batch' + str(j+1) + 'of' + str(len(val_dataloader))
        # 마저 필사

tensor([  101, 66889, 80858, 23833, 22035, 10944, 84889,   571,   117, 52326,
        96807, 16130, 18155,   566,   548, 20705, 19187, 42024, 17068, 27284,
        10944, 14500,   569, 21304, 21477, 12876, 21304,   579, 24942, 11059,
          553, 38659, 11059, 60449, 13465, 27079,   580,   591,   102, 52326,
        96807, 16130, 18155, 11263, 91039, 17068, 14187, 14500,   569, 21304,
        21477, 31614, 66889,   547, 38544,   571,   579, 24942, 11059,   553,
        38659, 17937,   591,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

# validation epoch 마저 필사하기
# https://www.kaggle.com/vbookshelf/basics-of-bert-and-xlm-roberta-pytorch#Section-3